CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [21]:
import numpy as np
import random
import pandas as pd

#!pip install pyomo
#import pyomo.environ as pyEnv
#!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import copy
import time 

adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjaceny_df/adjacency_df-n100-m400-run1-risky30-inf15_utku.csv"))
node_features_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/node_features/node_features-n100-m400-run1-risky30-inf15_utku.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2

#sets
setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()

#parameters
wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV  ]
ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
#Blocking_Capacity = int((n) / 10) 
Blocking_Capacity = int((n) / 10)*3

start=time.time()
#model
Math_Model = gp.Model("Math_Model")

#variables
Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

    # Mult[i] =  (1-Y[i])*(1-I[i])   (1)
for i in setV:
    Math_Model.addConstr(Multi[i], sense=GRB.EQUAL, rhs=(1-Yi[i])*(1-Ii[i]), name="Multi constraint".format(i))#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i])*(1-I[i]) <=  t[i]    for all i in S
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*Multi[i] , sense=GRB.LESS_EQUAL, rhs=ti[i-1],name="Birinci:Tresholdu geçen birey hasta olur.".format(i))
#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i]) >  t[i]*I[i]         for all i in S 
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*(1-Yi[i]) , sense=GRB.GREATER_EQUAL, rhs=ti[i-1]*Ii[i],name="İkinci:Tresholdu geçen birey hasta olur.".format(i))

#Σ(over i Y[i])<= c 
Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

#I[i]=1       for all i in V-S(P) 
for i in setP:
    Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))

    
    
Math_Model.write("out.lp")
# asd.computeIIS()
# asd.write("model.ilp")

Math_Model.ModelSense = GRB.MINIMIZE
Math_Model.optimize()

#for v in Math_Model.getVars():
    #if(v.x>= -0.0001):
        #print('%s %g' % (v.varName, v.x))

vars = Math_Model.getVars()

adjacency_df['Blocked']=np.zeros((n,1))

adjacency_df['Ending_Infected']=np.zeros((n,1))


for i in range(1,n+1):
    adjacency_df.at[i,'Blocked']=vars[i-1].x
    adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x

Objective_Value = Math_Model.ObjVal

#adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]

adjacency_df['Blocked'].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-OBJ'+str(Objective_Value)+".csv",index=False)
end=time.time()




Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 216 rows, 300 columns and 515 nonzeros
Model fingerprint: 0xb308da6c
Model has 270 quadratic constraints
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e-03, 1e+00]
  QLMatrix range   [2e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
  QRHS range       [3e-02, 1e+00]
Presolve removed 108 rows and 126 columns
Presolve time: 0.00s
Presolved: 359 rows, 395 columns, 1540 nonzeros
Variable types: 0 continuous, 395 integer (395 binary)
Found heuristic solution: objective 16.0000000
Found heuristic solution: objective 6.0000000

Root relaxation: objective 0.000000e+00, 232 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [22]:
end-start

0.6765894889831543

In [23]:
ati2=adjacency_df.loc[adjacency_df['Blocked']==1,setP].index

In [24]:
node_features_df.iloc[np.array(ati2)-1,:]

,IsInfected,IsRisky,map_for_dict,Betweenness,Degree_Cent,Closeness_cent,Eigenvector_cent,Degree,Shortest_to_Infected,Shortest_to_Risky,Total_Impact_Ratio
0,0.0,1.0,0,0.010752,0.161616,0.412500,0.093605,16,1,0,4.429467
3,0.0,0.0,3,0.010981,0.141414,0.402439,0.077663,14,1,1,6.134375
4,0.0,0.0,4,0.034278,0.262626,0.469194,0.152084,26,1,1,9.735294
5,0.0,1.0,5,0.025579,0.222222,0.447964,0.125324,22,1,0,8.412998
10,0.0,0.0,10,0.032585,0.303030,0.471429,0.198383,30,1,1,12.602906
16,0.0,0.0,16,0.027885,0.242424,0.445946,0.133614,24,1,1,9.397190
23,0.0,1.0,23,0.012511,0.141414,0.407407,0.071828,14,1,0,6.389180
28,0.0,1.0,28,0.016011,0.202020,0.428571,0.119495,20,2,0,6.972556
32,0.0,1.0,32,0.014573,0.181818,0.443946,0.118116,18,1,0,6.528757
36,0.0,1.0,36,0.003866,0.080808,0.360000,0.035470,8,2,0,3.061350


In [4]:
node_features_df.iloc[np.array(ati2)-1,:]

,IsInfected,IsRisky,map_for_dict,Betweenness,Degree_Cent,Closeness_cent,Eigenvector_cent,Degree,Shortest_to_Infected,Shortest_to_Risky,Total_Impact_Ratio
3,0.0,0.0,3,0.010981,0.141414,0.402439,0.077663,14,1,1,6.134375
5,0.0,1.0,5,0.025579,0.222222,0.447964,0.125324,22,1,0,8.412998
11,0.0,0.0,11,0.014163,0.161616,0.407407,0.095535,16,1,1,6.647059
15,0.0,0.0,15,0.025213,0.242424,0.456221,0.148072,24,1,1,8.045802
24,0.0,0.0,24,0.029926,0.222222,0.436123,0.122786,22,1,1,6.515188
30,0.0,0.0,30,0.012922,0.161616,0.409091,0.081204,16,1,1,5.914835
37,0.0,0.0,37,0.024091,0.222222,0.441964,0.116513,22,1,1,9.422311
39,0.0,1.0,39,0.014337,0.141414,0.402439,0.067758,14,1,0,4.628846
45,0.0,0.0,45,0.028916,0.262626,0.469194,0.169104,26,1,1,12.873016
56,0.0,0.0,56,0.006641,0.101010,0.368030,0.042036,10,1,1,4.415504


In [5]:
node_features_df.iloc[np.array(setP)-1,:].describe()

,IsInfected,IsRisky,map_for_dict,Betweenness,Degree_Cent,Closeness_cent,Eigenvector_cent,Degree,Shortest_to_Infected,Shortest_to_Risky,Total_Impact_Ratio
count,5.0,5.0,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.0,5.000000
mean,1.0,0.0,46.20000,0.012921,0.149495,0.406284,0.083085,14.800000,9999.0,0.0,5.291801
std,0.0,0.0,26.93882,0.008842,0.054583,0.034711,0.030583,5.403702,0.0,0.0,2.230177
min,1.0,0.0,20.00000,0.002376,0.080808,0.352313,0.039325,8.000000,9999.0,0.0,3.662050
25%,1.0,0.0,29.00000,0.005124,0.101010,0.391304,0.063887,10.000000,9999.0,0.0,4.000000
50%,1.0,0.0,33.00000,0.016583,0.181818,0.421277,0.097311,18.000000,9999.0,0.0,4.376430
75%,1.0,0.0,66.00000,0.017128,0.181818,0.432314,0.101615,18.000000,9999.0,0.0,5.290493
max,1.0,0.0,83.00000,0.023392,0.202020,0.434211,0.113285,20.000000,9999.0,0.0,9.130031


In [6]:
node_features_df['Degree'].sum()

1600

In [25]:
node_features_df.iloc[np.array(setP)-1,:].describe()

,IsInfected,IsRisky,map_for_dict,Betweenness,Degree_Cent,Closeness_cent,Eigenvector_cent,Degree,Shortest_to_Infected,Shortest_to_Risky,Total_Impact_Ratio
count,15.0,15.0,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.0,15.0,15.000000
mean,1.0,0.0,49.066667,0.013942,0.157576,0.411679,0.088608,15.600000,9999.0,0.0,5.452530
std,0.0,0.0,29.470243,0.008906,0.054100,0.032685,0.033095,5.355905,0.0,0.0,2.387748
min,1.0,0.0,1.000000,0.001401,0.060606,0.343750,0.026646,6.000000,9999.0,0.0,1.748116
25%,1.0,0.0,24.500000,0.005865,0.121212,0.397693,0.067417,12.000000,9999.0,0.0,3.904861
50%,1.0,0.0,55.000000,0.015792,0.161616,0.414226,0.096055,16.000000,9999.0,0.0,5.290493
75%,1.0,0.0,75.500000,0.019812,0.202020,0.433262,0.107450,20.000000,9999.0,0.0,6.584788
max,1.0,0.0,88.000000,0.030891,0.242424,0.454128,0.141682,24.000000,9999.0,0.0,9.521661


In [7]:
ati=adjacency_df['Blocked'].tolist()

In [8]:
adjacency_df['Blocked']==1

1      False
2      False
3      False
4       True
5      False
       ...  
96     False
97     False
98      True
99     False
100    False
Name: Blocked, Length: 100, dtype: bool

In [9]:
(adjacency_df.loc[adjacency_df['Ending_Infected']==1,'Blocked']).sum()

0.0

In [10]:
(adjacency_df.loc[adjacency_df['Ending_Infected']>0.1,'IsRisky']).sum()

0.0

In [11]:
len(adjacency_df)

100

In [12]:
adjacency_df.loc[adjacency_df['Blocked']==1,'IsRisky'].sum()

4.0

In [13]:
adjacency_df.loc[adjacency_df['Blocked']==1,setP]

,21,30,34,67,84
4,0.0,0.000,0.000,0.881,0.000
6,0.0,0.000,0.000,0.714,0.000
12,0.0,0.372,0.000,0.000,0.000
16,0.0,0.000,0.000,0.000,0.939
25,0.0,0.000,0.000,0.963,0.000
31,0.0,0.000,0.000,0.774,0.000
38,0.0,0.000,0.000,0.580,0.000
40,0.0,0.000,0.000,0.000,0.584
46,0.0,0.000,0.369,0.000,0.000
57,0.0,0.000,0.000,0.861,0.000


In [14]:
adjacency_df.loc[adjacency_df['Blocked']==1,setV]

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
4,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,...,0.000,0.000,0.000,0.617,0.000,0.000,0.000,0.0,0.0,0.0
6,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,...,0.995,0.000,0.000,0.000,0.857,0.000,0.000,0.0,0.0,0.0
12,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.138,0.000,...,0.000,0.813,0.442,0.000,0.000,0.000,0.000,0.0,0.0,0.0
16,0.0,0.000,0.0,0.657,0.0,0.000,0.000,0.0,0.000,0.000,...,0.000,0.965,0.000,0.161,0.000,0.000,0.000,0.0,0.0,0.0
25,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.709,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.0,0.0,0.0
31,0.0,0.000,0.0,0.000,0.0,0.000,0.023,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.793,0.0,0.0,0.0
38,0.0,0.008,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,...,0.948,0.791,0.000,0.000,0.000,0.757,0.000,0.0,0.0,0.0
40,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0
46,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.703,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0
57,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0


In [15]:
adjacency_df.loc[83,:]

1                  0.000
2                  0.000
3                  0.000
4                  0.000
5                  0.000
                   ...  
Threshold          0.909
IsInfected         0.000
IsRisky            0.000
Blocked           -0.000
Ending_Infected   -0.000
Name: 83, Length: 105, dtype: float64

In [16]:
adjacency_df.loc[adjacency_df['Blocked']==1,'Blocked']

4     1.0
6     1.0
12    1.0
16    1.0
25    1.0
31    1.0
38    1.0
40    1.0
46    1.0
57    1.0
64    1.0
66    1.0
91    1.0
98    1.0
Name: Blocked, dtype: float64